# Rivulet: U. S. NOAA CoastWatch and OceanWatch

In [ ]:
%pip install erddapy

## Part 0: Connect to API and Establish Custom Settings

In [ ]:
import pandas as pd
from erddapy import ERDDAP

In [ ]:
LAT_MIN = 24.0
LAT_MAX = 30.0
LON_MIN = -90.0
LON_MAX = -80.0

In [ ]:
TARGET_DATE_STR = '2025-09-15' 

## Part 1: Connect to CoastWatch to Obtain Recent Sea Level Data

Like the issue with AQS versus AirNow, NOAA CoastWatch data includes versions that feature near real-time data and versions that are delayed for quality control purposes.

We'll use a version that is quality controlled for scientific research here, but it is possible to access realtime data if you are interested in fetching a dataset that is related to a current event of interest by selecting a different DATASET_ID. Note that those data have not gone through the same rigor of validation processes. 

NOTE If you need data older than 2015, you should look for the "Reprocessed" or "Science Quality" dataset on the NOAA ERDDAP site, often labeled as noaacwSlaDaily or similar.

Visit https://coastwatch.noaa.gov/cw_html/cwViewer.html
For documentation about the API check out https://coastwatch.noaa.gov/erddap/index.html
A list of all datasets is here: https://coastwatch.noaa.gov/erddap/info/index.html?page=1&itemsPerPage=2000 (Note not all datasets are publicly accessible)

In [ ]:
# For this section accessing recent data, we want to use coastwatch

erddap_obj = ERDDAP(
    server='https://coastwatch.noaa.gov/erddap',
    protocol='griddap',
    response='nc'
)

In [ ]:
# Recommended by Gemini; REVSIT THIS
# The dataset IDs are mapped to measures in last column of dataset list above
DATASET_ID = 'noaacwBLENDEDsshDaily' 

In [ ]:
from datetime import datetime, timedelta

target_date = datetime.strptime(TARGET_DATE_STR, '%Y-%m-%d')

window_start = target_date + timedelta(days=5)
window_end = target_date - timedelta(days=5)

In [ ]:
print(f"Fetching data from {DATASET_ID}...")
print(f"Time: {window_start} to {window_end}")
print(f"Box: Lat({LAT_MIN}), Lon({LON_MAX})")
    
erddap_obj.dataset_id = DATASET_ID
    
# Set constraints
erddap_obj.griddap_initialize()

erddap_obj.constraints['time>='] = window_start
erddap_obj.constraints['time<='] = window_end
erddap_obj.constraints['latitude>='] = LAT_MIN
erddap_obj.constraints['latitude<='] = LAT_MAX
erddap_obj.constraints['longitude>='] = LON_MIN
erddap_obj.constraints['longitude<='] = LON_MAX

Select the data to fetch below. "sla" is sea level anomaly (the deviation from mean sea surface hieght).

In [ ]:
# Select variables to download (ssh/sla and coordinates)
erddap_obj.variables = ['sla']

In [ ]:
latest_sla_data = erddap_obj.to_pandas()

In [ ]:
latest_sla_data.head()

In [ ]:
latest_sla_data.to_csv('sealevel.csv')

## Part 2: Connect to OceanWatch to Obtain Historical Sea Level Data

Now let's fetch some data from 10 years prior to the target date for comparison. For data earlier than 2015, we need to use a different database, AOML, and the appropriate yearly dataset.

In [ ]:
erddap_obj = ERDDAP(
    server='https://erddap.aoml.noaa.gov/hdb/erddap', 
    protocol='griddap',
    response='nc'
)

# Set to the correct database for data 10 years earlier
ten_years_ago = target_date.year - 10

ten_years_earlier = target_date.replace(year=ten_years_ago)

window_start = ten_years_earlier + timedelta(days=5)
window_end = ten_years_earlier - timedelta(days=5)

DATASET_ID = f'SEA_SURFACE_HEIGHT_{str(ten_years_ago)}_v3'

In [ ]:
erddap_obj.dataset_id = DATASET_ID

erddap_obj.griddap_initialize()

erddap_obj.constraints['time>='] = window_start
erddap_obj.constraints['time<='] = window_end
erddap_obj.constraints['latitude>='] = LAT_MIN
erddap_obj.constraints['latitude<='] = LAT_MAX
erddap_obj.constraints['longitude>='] = LON_MIN
erddap_obj.constraints['longitude<='] = LON_MAX

ten_year_data = erddap_obj.to_pandas()
ten_year_data

The dataset starts in 1993, so for most recent-ish years, you can get data from 10 and 20 years ago. Below, we repeat the process one more time to fetch data from 20 years prior to the target date.

In [ ]:
twenty_years_ago = target_date.year - 20
erddap_obj.dataset_id = f'SEA_SURFACE_HEIGHT_{str(twenty_years_ago)}_v3'

twenty_years_earlier = target_date.replace(year=twenty_years_ago)

window_start = twenty_years_earlier + timedelta(days=5)
window_end = twenty_years_earlier - timedelta(days=5)

erddap_obj.griddap_initialize()

erddap_obj.constraints['time>='] = window_start
erddap_obj.constraints['time<='] = window_end
erddap_obj.constraints['latitude>='] = LAT_MIN
erddap_obj.constraints['latitude<='] = LAT_MAX
erddap_obj.constraints['longitude>='] = LON_MIN
erddap_obj.constraints['longitude<='] = LON_MAX

twenty_year_data = erddap_obj.to_pandas()
twenty_year_data

Let's merge all the datasets, focusing only on sla for now and ignoring all other attributes except latitude and datetime.

In [ ]:
def _extract_sla_df(df):
    cols = df.columns
    time_col = next(c for c in cols if 'time' in c.lower())
    lat_col = next(c for c in cols if 'lat' in c.lower())
    lon_col = next(c for c in cols if 'lon' in c.lower())
    sla_col = next(c for c in cols if 'sla' in c.lower())
    out = df[[time_col, lat_col, lon_col, sla_col]].copy()
    out.columns = ['datetime', 'latitude', 'longitude', 'sla']
    out['datetime'] = pd.to_datetime(out['datetime'])
    return out

merged_sla = pd.concat(
    [
        _extract_sla_df(latest_sla_data),
        _extract_sla_df(ten_year_data),
        _extract_sla_df(twenty_year_data),
    ],
    ignore_index=True
)

merged_sla

In [ ]:
merged_sla.to_csv('sealevelhistoric.csv')